In [13]:
!unzip ml-100k.zip


Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [14]:
import pandas as pd

# Load the ratings data
column_names = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=column_names)

# Display the first few rows
print(ratings.head())


   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596


In [15]:
# Create the user-item matrix
user_item_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

# Display the user-item matrix
print(user_item_matrix.head())


movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
5          4.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
5          0.0   0.0   0.0   0.0  

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Display the user similarity matrix
print(user_similarity_df.head())


user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.166931  0.047460  0.064358  0.378475  0.430239  0.440367   
2        0.166931  1.000000  0.110591  0.178121  0.072979  0.245843  0.107328   
3        0.047460  0.110591  1.000000  0.344151  0.021245  0.072415  0.066137   
4        0.064358  0.178121  0.344151  1.000000  0.031804  0.068044  0.091230   
5        0.378475  0.072979  0.021245  0.031804  1.000000  0.237286  0.373600   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.319072  0.078138  0.376544  ...  0.369527  0.119482  0.274876   
2        0.103344  0.161048  0.159862  ...  0.156986  0.307942  0.358789   
3        0.083060  0.061040  0.065151  ...  0.031875  0.042753  0.163829   
4        0.188060  0.101284  0.060859  ...  0.052107

In [17]:
def get_recommendations(user_id, num_recommendations=5):
    # Get similarity scores for the target user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)

    # Store recommended movies
    recommendations = []

    # Loop through similar users to find movies to recommend
    for similar_user in similar_users.index[1:]:  # skip the first entry (the user itself)
        # Get movies that the similar user has rated highly (4 or 5)
        highly_rated_movies = user_item_matrix.loc[similar_user][user_item_matrix.loc[similar_user] >= 4].index.tolist()

        # Filter out movies that the target user has already seen
        unseen_movies = [movie for movie in highly_rated_movies if user_item_matrix.loc[user_id, movie] == 0]

        # Add unseen movies to the recommendations list
        recommendations.extend(unseen_movies)

        # Stop if we've reached the desired number of recommendations
        if len(recommendations) >= num_recommendations:
            break

    return recommendations[:num_recommendations]

# Test the recommendation function for a sample user
user_id = 1
print(f"Recommendations for user {user_id}: {get_recommendations(user_id)}")


Recommendations for user 1: [276, 286, 317, 318, 382]


In [18]:
# Load the movie titles data
movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action',
                 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=movie_columns, encoding='latin-1', usecols=['movie_id', 'title'])

# Create a dictionary to map movie IDs to movie titles
movie_dict = pd.Series(movies.title.values, index=movies.movie_id).to_dict()


In [19]:
def get_recommendations(user_id, num_recommendations=5):
    # Get similarity scores for the target user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)

    # Store recommended movies
    recommendations = []

    # Loop through similar users to find movies to recommend
    for similar_user in similar_users.index[1:]:  # skip the first entry (the user itself)
        # Get movies that the similar user has rated highly (4 or 5)
        highly_rated_movies = user_item_matrix.loc[similar_user][user_item_matrix.loc[similar_user] >= 4].index.tolist()

        # Filter out movies that the target user has already seen
        unseen_movies = [movie for movie in highly_rated_movies if user_item_matrix.loc[user_id, movie] == 0]

        # Add unseen movies to the recommendations list
        recommendations.extend(unseen_movies)

        # Stop if we've reached the desired number of recommendations
        if len(recommendations) >= num_recommendations:
            break

    # Map movie IDs to titles
    recommendations_titles = [movie_dict[movie] for movie in recommendations[:num_recommendations]]

    # Print each recommended title on a new line
    print(f"Recommendations for user {user_id}:")
    for title in recommendations_titles:
        print(f"- {title}")

# Test the recommendation function with titles
user_id = 1
get_recommendations(user_id)


Recommendations for user 1:
- Leaving Las Vegas (1995)
- English Patient, The (1996)
- In the Name of the Father (1993)
- Schindler's List (1993)
- Adventures of Priscilla, Queen of the Desert, The (1994)
